In [1]:
import time
import re

import requests
import lxml.html
from pymongo import MongoClient

In [2]:
def main():
    """
    クローラーのメインの処理
    """
    
    client = MongoClient('localhost',27017) #ローカルホストのMongoDBに接続する
    collection = client.scraping.ebooks # scrapingデータベースのebooksコレクションを得る
    #データを一意に識別するキーを格納するkeyフィールドにユニークなインデックスを作成する。
    collection.create_index('key',unique=True)
    
    response = requests.get('https://gihyo.jp/dp') #一覧ページを取得する
    urls = scrape_list_page(response) #詳細ページのURLを得る
    for url in urls:
        key = extract_key(url) #URLからキーを取得する。
        
        ebook = collection.find_one({'key':key}) #MonogoDBからKeyに該当するデータを探す。
        if not ebook: #MongoDBに存在しない時だけ、詳細ページをクロールする。
            time.sleep(1)
            response = requests.get(url)
            ebook = scrape_detail_page(response)
            collection.insert_one(ebook) #電子書籍の情報をMongoDBに保存する。
            
        print(ebook) #電子書籍の情報を表示する

In [3]:
def scrape_list_page(response):
    """
    一覧ページのResponseから詳細ページのURLを抜き出す。
    """
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    
    for a in root.cssselect('#listBook a[itemprop="url"]'):
        url = a.get('href')
        yield url

In [4]:
def scrape_detail_page(response):
    """
    詳細ページのResponseから電子書籍の情報をdictで得る。
    """
    root = lxml.html.fromstring(response.content)
    ebook = {
        'url':response.url,#URL
        'key':extract_key(response.url), #URLから抜き出したキー
        'title':root.cssselect('#bookTitle')[0].text_content(), #タイトル
        'price':root.cssselect('.buy')[0].text.strip(), #価格
        'content':[normalize_spaces(h3.text_content()) for h3 in root.cssselect('#content > h3')],#目次
    }
    return ebook

In [5]:
def extract_key(url):
    """
    URLからキー(URLの末尾のISBN)を抜き出す。
    """
    m = re.search(r'/([^/]+)$',url)
    return m.group(1)

In [6]:
def normalize_spaces(s):
    """
    連続する空白を1つのスペースに置き換え、前後の空白は削除した新しい文字列を取得する。
    """
    return re.sub(r'\s+',' ',s)

In [7]:
if __name__=='__main__':
    main()

{'url': 'https://gihyo.jp/dp/ebook/2018/978-4-7741-9792-0', 'key': '978-4-7741-9792-0', 'title': 'WEB+DB PRESS Vol.104', 'price': '1,598円', 'content': [' 特集1 ［モダンなコードをギュッと凝縮！］イマドキPython入門 文法，機械学習，Web開発を一気に学ぼう ', ' 特集2 いきなりiPhoneアプリ開発 カメラの写真を加工してTwitter投稿！ ', ' 特集3 はじめてのUnity シューティングゲームを作ろう！ ', '連載', 'コラム', 'Special Report', '特別広報'], '_id': ObjectId('5ad0b833a43c3a0dd71cc230')}
{'url': 'https://gihyo.jp/dp/ebook/2018/978-4-7741-9791-3', 'key': '978-4-7741-9791-3', 'title': 'ゼロからはじめるLINE ライン スマートガイド 改訂版', 'price': '980円', 'content': ['第1章 LINEを始めよう', '第2章 トークや通話を楽しもう', '第3章 もっと友だちを追加しよう', '第4章 スタンプを入手しよう', '第5章 トークをもっと楽しもう', '第6章 グループを活用しよう', '第7章 LINE をもっと使いこなそう', '第8章 LINE を安心して使おう', '第9章 LINE の気になるQ&A'], '_id': ObjectId('5ad0b838a43c3a0dd71cc231')}
{'url': 'https://gihyo.jp/dp/ebook/2018/978-4-7741-9787-6', 'key': '978-4-7741-9787-6', 'title': '［改訂新版］C言語による標準アルゴリズム事典', 'price': '2,500円', 'content': [], '_id': ObjectId('5ad0b839a43c3a0dd71cc232')}
{'url': 'https://gihyo.jp/dp/ebook/201

{'url': 'https://gihyo.jp/dp/ebook/2018/978-4-7741-9733-3', 'key': '978-4-7741-9733-3', 'title': '基礎からわかる時系列分析―Rで実践するカルマンフィルタ・MCMC・粒子フィルタ―', 'price': '3,980円', 'content': ['第1章 はじめに', '第2章 確率・統計に関する基礎', '第3章 Rで時系列データを扱う際の基礎', '第4章 時系列分析ひとめぐり', '第5章 状態空間モデル', '第6章 状態空間モデルにおける状態の推定', '第7章 線形・ガウス型状態空間モデルの一括解法', '第8章 線形・ガウス型状態空間モデルの逐次解法', '第9章 線形・ガウス型状態空間モデルにおける代表的な成分モデルの紹介と分析例', '第10章 一般状態空間モデルの一括解法', '第11章 一般状態空間モデルの逐次解法', '第12章 一般状態空間モデルにおける応用的な分析例', '付録'], '_id': ObjectId('5ad0b857a43c3a0dd71cc244')}
{'url': 'https://gihyo.jp/dp/ebook/2018/978-4-7741-9731-9', 'key': '978-4-7741-9731-9', 'title': 'らくらく突破 運行管理者試験＜旅客＞合格教本', 'price': '2,080円', 'content': ['第1章 道路運送法', '第2章 道路運送車両法', '第3章 道路交通法', '第4章 労働基準法', '第5章 実務上の知識及び能力'], '_id': ObjectId('5ad0b858a43c3a0dd71cc245')}
{'url': 'https://gihyo.jp/dp/ebook/2018/978-4-7741-9729-6', 'key': '978-4-7741-9729-6', 'title': '株式会社＆合同会社のための 法人税の確定申告', 'price': '1,880円', 'content': ['Part 1 決算報告書と法人税申告書の基本', 'Part 2 法人税と法人住民税，事業税を計算する申告書の基本', 'Part 3